In [ ]:
# %%writefile flask_prefix_tree.py
from flask import Flask, request, jsonify, g
import json
import pandas as pd
import re

app = Flask(__name__)
class PrefixTree:
    def __init__(self):
        self.tree = [{}]
    
    def insert(self, key):
        '''
        Добавляет key в дерево
        '''
        if not self.check(key):
            mas = self.tree
            for i in key:
                if i in mas[0]:
                    mas = mas[0][i]
                else:
                    mas[0][i] = [{}]
                    mas = mas[0][i]
            mas.append(1)
    
    def check(self, key, check_only=True):
        '''Проверяет присутствует ли  строка в дереве. Если increase = True Увеличивает число обращаение по данной строке на 1'''
        mas = self.tree
        for i in key:
            if i not in mas[0]:
                return False
            else:
                mas = mas[0][i]
        if len(mas) != 1:
            mas[1] += 1
            return True if check_only else (True, mas[1])
        else:
            return False

    def check_part(self, key):
        '''Проверяет наличие подстроки в дереве'''
        mas = self.tree
        for i in key:
            if i not in mas[0]:
                return False
            else:
                mas = mas[0][i]
        return True 

    
    def top_5(self, key):
        
        def f(m, key):
            num = 0
            w = ''
            top = {}
            for i in m[0]:
                if self.check(key + i) and len(m[0][i]) > 1:
                    num = m[0][i][1]
                    w = key + i
                temp = f(m[0][i], key + i)
                if temp != (0, ''):
                    d[temp[1]] = temp[0]
            return num, w
        
        m = self.tree
        d = dict()
        for i in key:
            m = m[0][i]
        if self.check(key):
            d[key] = m[1]
        f(m, key)
        top = [(v, k) for k, v in d.items()]
        top.sort(reverse=True)
        guess = [i[1] for i in top[:5]]
        return guess
    
def add_links(word, df):
    '''Выводит 3 наиболее популярные ссылки на статьи, содержащие в названии введенное слово'''
    i = k = 0
    links = []
    while i < df.shape[0] and k < 5:
        if word in df.Title.iloc[i]:
            links.append(df.Link.iloc[i])
            k += 1
        i += 1
    return links
            
            
def init_prefix_tree(filename):
    #TODO в данном методе загружаем данные из файла. Предположим вормат файла "Строка, чтобы положить в дерево" \t "json значение для ноды" \t частота встречаемости
    filename = 'data'
    data = pd.read_csv(filename, sep="@")
    baobab = PrefixTree()
    for row in data.Title:
        items = re.findall('[a-z]+', row) # Оставляем только буквы
        for item in items:
            if len(item) > 4:
                baobab.insert(item.lower())
    return baobab, data

# def t(string):
#     tree, df = init_prefix_tree('data')
#     res = []
#     words = tree.top_5(string)
#     guess = { w : add_links(w, df) for w in words}
#     return guess

@app.route("/get_sudgest/<string>", methods=['GET', 'POST'])
def return_sudgest(string):
    #TODO по запросу string вернуть json, c топ-10 саджестами, и значениями из нод  
    tree, df = init_prefix_tree('data')
    try:
        words = tree.top_5(string)
        print(words)
        guess = { w : add_links(w, df) for w in words}
    except: 
        guess = "Nothing was found"
    return json.dumps(guess)

@app.route("/")
def hello():
    return 'Hi there. Try ~/get_sudgest/<string>'
    
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Nov/2018 02:42:31] "GET /get_sudgest/quantum HTTP/1.1" 200 -


['quantum']


In [155]:
pr_tree = PrefixTree()
pr_tree.insert("строка")
assert pr_tree.check("строка")
assert not pr_tree.check("строк")
assert not pr_tree.check("строка1")
assert pr_tree.check_part("стро")

In [156]:
g = init_prefix_tree('data')

In [157]:
g[0]

In [159]:
g[0].top_5('quantum')

['quantum']

In [150]:
t('quantum')

TypeError: 'in <string>' requires string as left operand, not tuple

In [44]:
df = pd.read_csv('data', sep='@')
df = df.sort_values('Qoutes',ascending=False)

In [151]:
word = 'quantum'.lower()
def add_links(word,df):
    '''
    Выводит 3 наиболее популярные ссылки на статьи, содержащие в названии введенное слово
    '''
    i = k = 0
    links = []
    while i < df.shape[0] and k < 5:
        if word in df.Title.iloc[i]:
            links.append(df.Link.iloc[i])
            k += 1
        i += 1
    return links

In [153]:
{w: add_links('quantum',df) for w in word}

{'q': ['https://elibrary.ru/item.asp?id=27566821',
  'https://elibrary.ru/item.asp?id=24529250',
  'https://elibrary.ru/item.asp?id=24712232',
  'https://elibrary.ru/item.asp?id=24521905',
  'https://elibrary.ru/item.asp?id=24544646'],
 'u': ['https://elibrary.ru/item.asp?id=27566821',
  'https://elibrary.ru/item.asp?id=24529250',
  'https://elibrary.ru/item.asp?id=24712232',
  'https://elibrary.ru/item.asp?id=24521905',
  'https://elibrary.ru/item.asp?id=24544646'],
 'a': ['https://elibrary.ru/item.asp?id=27566821',
  'https://elibrary.ru/item.asp?id=24529250',
  'https://elibrary.ru/item.asp?id=24712232',
  'https://elibrary.ru/item.asp?id=24521905',
  'https://elibrary.ru/item.asp?id=24544646'],
 'n': ['https://elibrary.ru/item.asp?id=27566821',
  'https://elibrary.ru/item.asp?id=24529250',
  'https://elibrary.ru/item.asp?id=24712232',
  'https://elibrary.ru/item.asp?id=24521905',
  'https://elibrary.ru/item.asp?id=24544646'],
 't': ['https://elibrary.ru/item.asp?id=27566821',
  'ht

In [ ]:
df.shape[0]